# Prime Factoring by QUBO
Now we solve simple prime factoring by using QUBO matrix to solve the minimum problem.

## Factoring 15
Now we try factoring 15 to p and q.
p and q are prime factors

$$
15 = p*q
$$

To get p and q we solve the equation below.

$$
E=(15 - p*q)^2
$$

p and q also can write as decimal number using qubits which has binary value.

$$
p = 1+2q_0+4q_1 \\
q = 1+2q2
$$

substitute these and we get equation below.

$$
E = \{15-(1+2q_0+4q_1)(1+2q_2)\}^2
$$

In [1]:
from sympy import *

q0 = Symbol('q0')
q1 = Symbol('q1')
q2 = Symbol('q2')

E = (15-(1+2*q0+4*q1)*(1+2*q2))**2
E = expand(E)
E

16*q0**2*q2**2 + 16*q0**2*q2 + 4*q0**2 + 64*q0*q1*q2**2 + 64*q0*q1*q2 + 16*q0*q1 + 16*q0*q2**2 - 104*q0*q2 - 56*q0 + 64*q1**2*q2**2 + 64*q1**2*q2 + 16*q1**2 + 32*q1*q2**2 - 208*q1*q2 - 112*q1 + 4*q2**2 - 56*q2 + 196

as the binary numer rules we get.

In [2]:
E = E.subs(q0**2, q0)
E = E.subs(q1**2, q1)
E = E.subs(q2**2, q2)
E

128*q0*q1*q2 + 16*q0*q1 - 56*q0*q2 - 52*q0 - 48*q1*q2 - 96*q1 - 52*q2 + 196

## Solve the QUBO with QAOA
By using QAOA we can easily solve the upper problem.

In [12]:
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

hamiltonian = 128*q(0)*q(1)*q(2)+16*q(0)*q(1)-56*q(0)*q(2)-52*q(0)-48*q(1)*q(2)-96*q(1)-52*q(2)
step = 16

result = vqe.Vqe(vqe.QaoaAnsatz(hamiltonian, step)).run()
print(result.most_common(12))

(((0, 1, 1), 0.842337256187905), ((1, 1, 1), 0.13083477998166643), ((1, 1, 0), 0.015503198595644531), ((1, 0, 1), 0.007599466796274913), ((1, 0, 0), 0.002139956732585671), ((0, 0, 1), 0.0007482531711899896), ((0, 0, 0), 0.0007311234439760303), ((0, 1, 0), 0.00010596509075921791))


Now we get (0,1,1) and it shows

$$
p = 1+2*0+4*1 = 5 \\
q = 1+2*1 = 3
$$

And finally we solved the problem using QUBO and minimum problem to solve simple factoring.